<a href="https://colab.research.google.com/github/alexontour/snippets/blob/main/snip_fhir_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Author: Alexander Kollmann, 09/2022**

---

**Funktion**

FHIR - Abfragen und Suchen (nativ und mittels fhirclient)

---



**Referenzen**

https://github.com/Alvearie/FHIR-from-Jupyter 

https://learn.microsoft.com/en-us/azure/healthcare-apis/fhir/search-samples

https://cloud.google.com/healthcare-api/docs/how-tos/fhir-advanced-search

https://colab.research.google.com/drive/1OSuqqACnCqw8h67E7DKDMyIGO1C2qbEq?usp=sharing



---


# Neuer Abschnitt

In [1]:
import json
import requests
from collections import OrderedDict
from io import StringIO
from IPython.display import IFrame

In [3]:
# Define Base URL
url = "http://hapi.fhir.org/baseR4/" # Open HAPI FHIR Server
#url = "https://server.fire.ly/R4/" # Open Firely Server
headers = {"Content-Type": "application/fhir+json;charset=utf-8"}

In [ ]:
# now add some search parameters

# each FHIR resource type has its own set of parameters; find them toward the bottom of the page for that resource type in the specification
# for example, for the Patient resource type, see https://www.hl7.org/fhir/patient.html#search
IFrame('https://www.hl7.org/fhir/patient.html#search', width=1200, height=500)

# Einfache Suche

In [ ]:
# Abfrage nach Name UND Geburtsdatum
# Erwartetes Ergebnis: 1 Treffer

req = url + "Patient?" + "birthdate" + "=" + "1950-01-01" + "&" + "name" + "=" + "MrXXX"
print(req)
response = requests.request("GET", req, headers=headers)
result = response.json()
patient_id = result['entry'][0]['resource']['id']
print(response.text)

In [65]:
patient_id

'7168516'

In [ ]:
# Abfrage der Condition zu einem Patienten

# Teatpatient: MrXXX
#patient_id = str(7168516)
# Get Condition of this patient
req = url + "Condition?" + "patient=" + patient_id
response = requests.request("GET", req, headers=headers)
print("Anzahl: " + str(json.loads(response.text)['total']))
print(response.text)

In [ ]:
# Search Observations with the term ‘Cholesterol’’ in text description.

req = url + "Observation?_text=Cholesterol"
response = requests.request("GET", req, headers=headers)
print(response.text)

In [ ]:
# alle Patienten mit Namen
req = url + "Patient?name=Alex&_summary=count"
response = requests.request("GET", req, headers=headers)
print("Anzahl der Patienten: " + str(response.json().get('total')))

req = url + "Patient?name=Alex"
response = requests.request("GET", req, headers=headers)
print(response.text)

In [26]:
# Anzahl der Patienten
req = url + "Patient?_summary=count"
response = requests.request("GET", req, headers=headers)
print("Anzahl der Patienten: " + str(response.json().get('total')))
print(response.text)

Anzahl der Patienten: 301527
{
  "resourceType": "Bundle",
  "id": "4fd8da95-36a3-488a-b91c-0ace45b07993",
  "meta": {
    "lastUpdated": "2022-12-04T20:18:13.995+00:00",
    "tag": [ {
      "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
      "code": "SUBSETTED",
      "display": "Resource encoded in summary mode"
    } ]
  },
  "type": "searchset",
  "total": 301527
}


In [ ]:
# Anzahl der Patienten
req = url + "Patient?_total=accurate"
response = requests.request("GET", req, headers=headers)
print(response.text)

In [ ]:
# Rückgabe von Summary-Paramtern einer Ressource (idF. Patient)
req = url + "Patient?_summary=true"
# Rückgabe von ausgewählten Attributen einer Ressource (idF. Ressource Patient mit den Attributen id und gender)
req = url + "Patient?_elements=id,gender"
response = requests.request("GET", req, headers=headers)
print(response.text)

In [33]:
# Anzahl der Patienten ohne GebDat
req = url + "Patient?birthdate:missing&_summary=count"

#Anzahl der Patienten mit Sterbekennzeichen
req = url + "Patient?deceased:true&_summary=count"
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "5125880b-8725-4813-bf0f-ce66f5e4f8e4",
  "meta": {
    "lastUpdated": "2022-12-04T20:23:43.656+00:00",
    "tag": [ {
      "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
      "code": "SUBSETTED",
      "display": "Resource encoded in summary mode"
    } ]
  },
  "type": "searchset",
  "total": 301527
}


In [32]:
# Anzahl der Geschlechter und Abfragedauer
req = url + "Patient" + '?' + 'gender=male' + '&' + '_summary=count'
response = requests.request("GET", req, headers=headers)
print("male: " + str(response.json().get('total')) + " (" + str(response.elapsed.total_seconds()) + ' s)')
req = url + "Patient" + '?' + 'gender=female' + '&' + '_summary=count'
response = requests.request("GET", req, headers=headers)
print("female: " + str(response.json().get('total')) + " (" + str(response.elapsed.total_seconds()) + ' s)')
req = url + "Patient" + '?' + 'gender:missing=true' + '&' + '_summary=count'
response = requests.request("GET", req, headers=headers)
print("missing: " + str(response.json().get('total')) + " (" + str(response.elapsed.total_seconds()) + ' s)')

male: 143859 (7.192599 s)
female: 66509 (2.897513 s)
missing: None (20.498127 s)


In [30]:
# String Search
req = url + 'Patient' + '?' + 'family=Koll' + '&_elements=name'
#req = url + 'Patient' + '?' + 'family:exact=MrXXX' + '&_elements=name'
#req = url + 'Patient' + '?' + 'family:contains=Mr' + '&_elements=name'
response = requests.request("GET", req, headers=headers)
#print(response.text)

for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(', '.join(map(lambda n: n.get('family'), resource.get('name'))))

2376392: Kölle
1709914: Kollege
7167994: Koller
7170339: Koller
7162763: Kolli
7162765: Kolli
7162767: Kolli
7162769: Kolli
7162771: Kolli
7162773: Kolli
7162777: Kolli
7162779: Kolli
7162781: Kolli
7162783: Kolli
7162785: Kolli
7162787: Kolli
7162789: Kolli
7162915: Kolli
7163145: Kolli
7163147: Kolli


In [31]:
IFrame('https://www.hl7.org/fhir/search.html#prefix', width=1200, height=500)

In [ ]:
# Date Search
# date searches support lt(<), le(<=), gt(>), ge(>=), sa(starts after), and eb(ends before) "prefixes"

# Patienten mit Geburtsjahr
#req = url + 'Patient' + '?' + 'birthdate=1984'

# Patienten mit Geburtstag zwischen zwei Zeitpunkten
req = url + 'Patient?birthdate=gt2016-01-01&birthdate=lt2019-01-01'
response = requests.request("GET", req, headers=headers)
print(response.text)

In [ ]:
IFrame('https://build.fhir.org/observation.html#search', width=1200, height=500)

In [ ]:
# Range Search

code = "8867-4"

req = url + "/Observation?code=" + code + "&value-quantity=ge50&value-quantity=le200"
response = requests.request("GET", req, headers=headers)
print(response.text)

In [48]:
# Reference search
# Alle Patienten zurückgeben, die einen GP haben und den Namen/ Org des GP

req = url + 'Patient?general-practitioner:missing=false&_elements=id,name,generalPractitioner,managingOrganization'
response = requests.request("GET", req, headers=headers)
print(response.text)


{
  "resourceType": "Bundle",
  "id": "4b8647e7-68aa-4837-b533-5e54aef14ad9",
  "meta": {
    "lastUpdated": "2022-12-04T20:49:15.711+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Patient?_elements=id%2Cname%2CgeneralPractitioner%2CmanagingOrganization&general-practitioner%3Amissing=false"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=4b8647e7-68aa-4837-b533-5e54aef14ad9&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset&_elements=generalPractitioner,id,managingOrganization,name"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Patient/7176618",
    "resource": {
      "resourceType": "Patient",
      "id": "7176618",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2022-12-02T16:51:55.077+00:00",
        "source": "#8xHFTaaZnnkjIOzA",
        "tag": [ {
          "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
  

In [ ]:
#req = url + "Patient$export"

In [ ]:
# Alle Conditions zu einem Patienten

req = url + 'Condition' + '?' + 'patient=' + url + '/Patient/' + patient_id + '&_elements=code'
response = requests.request("GET", req, headers=headers)
print(response.text)

# Chained search

In [ ]:
req = url + 'Encounter?subject:Patient.name=Koll'
response = requests.request("GET", req, headers=headers)
print(str(response.text))

In [ ]:
req = url + 'Condition' + '?' + 'code=http://snomed.info/sct|44054006' + '&' + 'patient:Patient.gender=female&_summary=count'
response = requests.request("GET", req, headers=headers)
print(str(response.text))

In [ ]:
req = url + 'DiagnosticReport?subject:Patient.name=Alex'
response = requests.request("GET", req, headers=headers)
print(str(response.text))

In [57]:
# Alle Diagnosen von Patienten mit Geburtsjahr 1930
req = url + 'Condition?patient.birthdate=1930'
response = requests.request("GET", req, headers=headers)
print(str(response.text))

{
  "resourceType": "Bundle",
  "id": "b5418647-f6d6-4349-9491-643def069e54",
  "meta": {
    "lastUpdated": "2022-12-04T21:03:27.429+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Condition?patient.birthdate=1930"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=b5418647-f6d6-4349-9491-643def069e54&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Condition/6966594",
    "resource": {
      "resourceType": "Condition",
      "id": "6966594",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2022-08-22T21:18:20.961+00:00",
        "source": "#wIhXIGKf5Hsjs3df"
      },
      "code": {
        "coding": [ {
          "system": "http://snomed.info/sct",
          "code": "73211009",
          "display": "Diabetes mellitus (disorder)"
        } ],
        "text": "Facture of radius"
      },
  

# Reverse chained search

In [ ]:
# Search for all Patients with Diabetes Mellitus

req = url + 'Patient' + '?' + '_has:Condition:patient:code=http://snomed.info/sct|44054006' + '&_summary=count'
response = requests.request("GET", req, headers=headers)
print(response.text)

In [ ]:
# Anzahl der Patienten mit einer Observation "Heart Rate"
# https://loinc.org/8867-4/

code = "8867-4"

req = url + "Patient?_has:Observation:patient:code=" + code + "&_summary=count"
response = requests.request("GET", req, headers=headers)
print("Anzahl der Patienten: " + str(response.json().get('total')))
print(response.text)

In [ ]:
# Alle Patienten zurück geben mit einer gewissen Observation (idF. Heart Rate)
# Wenn es viele Treffer gibt, dann wird "Pageing" aktiviert.

req = url + "Patient?_has:Observation:patient:code=8867-4"

# die ersten 5 Ergebnisse zurückgebe
req = url + "Patient?_has:Observation:patient:code=8867-4&_count=5&_sort=-_lastUpdated"


response = requests.request("GET", req, headers=headers)
print(response.text)

# Include

In [ ]:
# _include lets you search for resource instances and include in the results other resources referenced by the target resource instances. 

req = url + 'Condition?code=http://snomed.info/sct|44054006' + '&' + '_include=Condition:patient'
response = requests.request("GET", req, headers=headers)
print(response.text)

# Reverse Include

In [69]:
#_revinclude allows you to search for resource instances and include in the results other resources that reference the target resource instances. 
req = url + 'Patient?gender=female&birthdate=1920-06' + '&' + '_revinclude=Condition:patient'
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "3b95fbfc-5641-45f7-89a7-98fa429f58d5",
  "meta": {
    "lastUpdated": "2022-12-04T21:24:41.874+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Patient?_revinclude=Condition%3Apatient&birthdate=1920-06&gender=female"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=3b95fbfc-5641-45f7-89a7-98fa429f58d5&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Patient/534645",
    "resource": {
      "resourceType": "Patient",
      "id": "534645",
      "meta": {
        "versionId": "2",
        "lastUpdated": "2020-03-27T15:34:48.987+00:00",
        "source": "#IsZ2AbcTKQ3C8Svj"
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">[FILTERED] <b>[FILTERED] </b></div><table class=\"hapiPr

# AddOn

In [ ]:
# Daten exportieren
req = url + '$export' + '?' + '_type=Patient,Condition'
response = requests.request("GET", req, headers=headers)
print(response.text)

In [ ]:
# Konfiguration des FHIR-Servers abrufen
req = url + 'metadata'
response = requests.request("GET", req, headers=headers)
print(response.text)

In [26]:
import pandas as pd

# Ergebnis (JSON) in Pandas Dataframe konvertieren
req = url + 'Patient' + '?' + '_has:Condition:patient:code=http://snomed.info/sct|44054006&_elements=id,gender,birthDate'
response = requests.request("GET", req, headers=headers)
#print(response.text)

data = json.loads(response.text)

df = pd.json_normalize(data, 'entry')
df.describe()

,fullUrl,resource.resourceType,resource.id,resource.meta.versionId,resource.meta.lastUpdated,resource.meta.source,resource.meta.tag,resource.gender,resource.birthDate,search.mode,resource.meta.profile
count,20,20,20,20,20,20,20,20,20,20,3
unique,20,1,20,4,20,20,1,2,8,1,1
top,https://hapi.fhir.org/baseR4/Patient/601740,Patient,601740,1,2020-02-07T11:03:45.714+00:00,#Io7DEVTOm0wT6Mkc,[{'system': 'http://terminology.hl7.org/CodeSy...,male,1990-11-24,match,[https://www.fhir.philips.com/4.0/StructureDef...
freq,1,20,1,16,1,1,20,17,7,20,3
